# 02. 소프트맥스 회귀(Softmax Regression) 이해하기

### 1. 로지스틱 회귀
시그모이드 함수는 예측값을 0과 1 사이의 값으로 만듭니다. 예를 들어 스팸 메일 분류기를 로지스틱 회귀로 구현하였을 때, 출력이 0.75이라면 이는 이메일이 스팸일 확률이 75%라는 의미가 됩니다. 반대로, 스팸 메일이 아닐 확률은 25%가 됩니다. 이 두 확률의 총 합은 1입니다.



### 2. 소프트맥스 회귀 = 다항 로지스틱 회귀(Multinomial Logistic Regression) 
확률의 총 합이 1이 되는 이 아이디어를 다중 클래스 분류 문제에 적용



### 2. 소프트맥스 함수(Softmax function)
소프트맥스 함수는 분류해야하는 정답지(클래스)의 총 개수를 k라고 할 때, k차원의 벡터를 입력받아 각 클래스에 대한 확률을 추정합니다

실제값을 원핫인코딩으로 표현하여 실제값과 가장 오차가 적은 값이 정답으로 한다. 

X = 5X4, Y' = 5X3, W = 4X3(X와 Y'의 곱), B=5X3(Y'와 동일)



Y' = softmax(XW + B)


# 03. 소프트맥스 회귀의 비용 함수 구현하기

### 1. 파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨)

In [13]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

# 비용함수
z = torch.rand(3,5, requires_grad=True)
hypothesis = F.softmax(z, dim=1) # 두번째 차원에 적용-> (3,5)에서 (,5)부분에 적용!
print(hypothesis)

# 임의의 레이블을 만들어주기
y = torch.randint(5,(3,)).long() # 0~4까지 숫자를 (3,)에 부여
print(y)

# 각 레이블에 대해서 원핫인코딩 수행
y_one_hot = torch.zeros_like(hypothesis) #모든 원소가 0의 값을 가진 3x5 텐서를 만든다
y_one_hot.scatter_(1,y.unsqueeze(1),1) 
# 연산 뒤에 _를 붙이면 In-place Operation (덮어쓰기 연산)

# torch.scatter(input, dim, index, src) → Tensor

# y_one_hot.scatter_(dim=1, (3x1 텐서), 1을 부여)

print(y.unsqueeze(1))

print(y_one_hot)
'''
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
'''

# 소프트맥스 회귀의 비용 함수
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

#2. 파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨)


print(cost)

tensor([0.0900, 0.2447, 0.6652])
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)
tensor([0, 2, 1])
tensor([[0],
        [2],
        [1]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
tensor(1.4689, grad_fn=<MeanBackward0>)


### 2. 파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨)

**1. F.softmax() + torch.log() = F.log_softmax()**

In [14]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [15]:
# High level -> 파이토치에서는 두 개의 함수를 결합한 F.log_softmax()도구를 제공한다
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

**2. F.log_softmax() + F.nll_loss() = F.cross_entropy()**

In [16]:
# Low level
# 첫번째 수식

print(y_one_hot)
'''
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
'''
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


tensor(1.4689, grad_fn=<MeanBackward0>)

In [17]:
# 두번째 수식
(y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [18]:
# F.nll_loss()를 사용할 때는 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용
# High level
# 세번째 수식
F.nll_loss(F.log_softmax(z, dim=1), y)

# nll = Negative Log Likelihood

# nll_loss는 F.log_softmax()를 수행한 후에 남은 수식들을 수행

# F.cross_entropy()는 F.log_softmax()와 F.nll_loss()를 포함하고 있다.


tensor(1.4689, grad_fn=<NllLossBackward>)

In [19]:
# 네번째 수식
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

# 04. 소프트맥스 회귀 구현하기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0] # 3개의 클래스 존재
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)